In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
import seaborn as sns
import datetime
import csv

%matplotlib inline

In [2]:
df = pd.read_csv('ML_Bond_metadata_corrected_dates.csv', header=0, parse_dates=['issue date',
                                                        'ratingAgency1EffectiveDate', 'ratingAgency2EffectiveDate'])
data = pd.read_csv('dataset.csv', header=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17261 entries, 0 to 17260
Data columns (total 24 columns):
isin                          17261 non-null object
issuer                        17261 non-null int64
issue date                    17260 non-null datetime64[ns]
Market                        17261 non-null int64
amtIssued                     17261 non-null int64
amtOutstanding                17261 non-null int64
collateralType                17261 non-null int64
coupon                        17261 non-null float64
couponFrequency               16553 non-null float64
couponType                    17261 non-null int64
industryGroup                 17261 non-null int64
industrySector                17261 non-null int64
industrySubgroup              17261 non-null int64
maturity                      17023 non-null object
maturityType                  17261 non-null int64
securityType                  17261 non-null int64
paymentRank                   17261 non-null int64
144aFlag 

In [4]:
df.head(10)

,isin,issuer,issue date,Market,amtIssued,amtOutstanding,collateralType,coupon,couponFrequency,couponType,...,maturityType,securityType,paymentRank,144aFlag,ratingAgency1Rating,ratingAgency1Watch,ratingAgency1EffectiveDate,ratingAgency2Rating,ratingAgency2Watch,ratingAgency2EffectiveDate
0,isin0,490,2011-08-12,2,183260000,183260000,19,10.000,2.0,1,...,5,3,3,0,0,0,NaT,0,0,NaT
1,isin1,1193,2010-10-09,2,261800000,198970000,26,5.500,2.0,1,...,10,5,8,0,0,0,NaT,0,0,NaT
2,isin2,2943,2014-11-14,2,78540000,78540000,28,5.750,2.0,1,...,10,5,9,1,0,0,NaT,0,0,NaT
3,isin3,3707,2013-04-03,1,1570800000,1570800000,28,4.750,1.0,6,...,5,2,9,0,18,0,2016-03-16,22,0,2013-02-05
4,isin4,3711,2014-11-25,0,1309000000,1309000000,12,7.125,1.0,6,...,13,3,5,0,15,0,2016-03-16,0,0,NaT
5,isin5,3711,2014-11-25,0,1309000000,1309000000,12,7.000,1.0,6,...,13,3,5,0,15,0,2016-03-16,0,0,NaT
6,isin6,3278,2012-12-17,1,523600000,523600000,5,1.500,1.0,1,...,0,2,8,0,5,0,2012-12-24,1,0,2014-07-11
7,isin7,2238,1985-03-09,1,523600000,135720000,12,1.000,4.0,3,...,13,3,5,0,19,0,2015-03-19,16,0,2014-11-02
8,isin8,404,2013-11-25,0,837760000,772920000,22,7.375,2.0,1,...,5,5,2,1,16,0,2014-04-17,0,0,NaT
9,isin9,1167,2014-02-27,1,418880000,418880000,22,2.803,2.0,1,...,0,5,2,1,19,0,2014-03-27,16,0,2014-07-03


In [5]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [6]:
start_date = date(2016, 3, 16)
end_date = date(2016, 6, 10)
for single_date in daterange(start_date, end_date):
    df[single_date.strftime("%d%b%Y")+'_B'] = np.zeros(df.shape[0])
    df[single_date.strftime("%d%b%Y")+'_S'] = np.zeros(df.shape[0])
df.head(10)

,isin,issuer,issue date,Market,amtIssued,amtOutstanding,collateralType,coupon,couponFrequency,couponType,...,05Jun2016_B,05Jun2016_S,06Jun2016_B,06Jun2016_S,07Jun2016_B,07Jun2016_S,08Jun2016_B,08Jun2016_S,09Jun2016_B,09Jun2016_S
0,isin0,490,2011-08-12,2,183260000,183260000,19,10.000,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,isin1,1193,2010-10-09,2,261800000,198970000,26,5.500,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,isin2,2943,2014-11-14,2,78540000,78540000,28,5.750,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,isin3,3707,2013-04-03,1,1570800000,1570800000,28,4.750,1.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,isin4,3711,2014-11-25,0,1309000000,1309000000,12,7.125,1.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,isin5,3711,2014-11-25,0,1309000000,1309000000,12,7.000,1.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,isin6,3278,2012-12-17,1,523600000,523600000,5,1.500,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,isin7,2238,1985-03-09,1,523600000,135720000,12,1.000,4.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,isin8,404,2013-11-25,0,837760000,772920000,22,7.375,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,isin9,1167,2014-02-27,1,418880000,418880000,22,2.803,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
i=0
while i<data.shape[0]:
    #print i
    j=i+1
    while j<data.shape[0] and data['isin'][j]==data['isin'][i]:
        if j%100000==0:
            print j
        j=j+1
    for k in range(i, j):
        if data['date'][k][0] == ' ':
            x = '0' + data['date'][k][1:]
        else:
            x = data['date'][k]
        df.loc[df['isin']==data['isin'][i], x + '_' + data['side'][k]] += data['volume'][k]
    i=j

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000


In [166]:
#df.to_csv('data.csv')
df = pd.read_csv('data.csv', header=0)
df.head(30)

,isin,issuer,issue date,Market,amtIssued,amtOutstanding,issued2,outstanding2,collateralType,collateral2,...,05Jun2016_B,05Jun2016_S,06Jun2016_B,06Jun2016_S,07Jun2016_B,07Jun2016_S,08Jun2016_B,08Jun2016_S,09Jun2016_B,09Jun2016_S
0,isin0,490,2011-08-12,2,183260000,183260000,3.358423e+16,3.358423e+16,19,361,...,0,0,0,0,0,0,0,0,0,0
1,isin1,1193,2010-10-09,2,261800000,198970000,6.853924e+16,3.958906e+16,26,676,...,0,0,0,0,0,0,0,0,100000,50000
2,isin2,2943,2014-11-14,2,78540000,78540000,6.168532e+15,6.168532e+15,28,784,...,0,0,0,0,0,0,0,0,0,0
3,isin3,3707,2013-04-03,1,1570800000,1570800000,2.467413e+18,2.467413e+18,28,784,...,0,0,0,0,310000,0,210000,0,50000,50000
4,isin4,3711,2014-11-25,0,1309000000,1309000000,1.713481e+18,1.713481e+18,12,144,...,0,0,0,0,0,0,0,0,0,0
5,isin5,3711,2014-11-25,0,1309000000,1309000000,1.713481e+18,1.713481e+18,12,144,...,0,0,0,0,0,0,0,0,0,0
6,isin6,3278,2012-12-17,1,523600000,523600000,2.741570e+17,2.741570e+17,5,25,...,0,0,120000,3990000,0,0,0,0,0,0
7,isin7,2238,1985-03-09,1,523600000,135720000,2.741570e+17,1.841992e+16,12,144,...,0,0,0,0,0,0,0,0,0,0
8,isin8,404,2013-11-25,0,837760000,772920000,7.018418e+17,5.974053e+17,22,484,...,0,0,100000,0,210000,0,0,0,0,0
9,isin9,1167,2014-02-27,1,418880000,418880000,1.754605e+17,1.754605e+17,22,484,...,0,0,0,0,0,0,0,0,0,0


In [165]:
#df['rating12'] = df['ratingAgency1Rating']*df['ratingAgency1Rating']
df['collateral2'] = df['collateralType']*df['collateralType']
df.to_csv('data.csv')

In [171]:
train = df.copy()
del train['issue date'], train['ratingAgency1EffectiveDate'], train['ratingAgency2EffectiveDate']
del train['couponFrequency'], train['maturity'], train['couponType']
train_b = train.ix[:, 1:24].copy()
train_s = train.ix[:, 1:24].copy()
start_date = date(2016, 3, 16)
end_date = date(2016, 6, 9)
for single_date in daterange(start_date, end_date):
    train_b[single_date.strftime("%d%b%Y")+'_B'] = train[single_date.strftime("%d%b%Y")+'_B']
    train_s[single_date.strftime("%d%b%Y")+'_S'] = train[single_date.strftime("%d%b%Y")+'_S']
    
test_b = train.ix[:, 1:24].copy()
test_s = train.ix[:, 1:24].copy()
start_date = date(2016, 3, 17)
end_date = date(2016, 6, 10)
for single_date in daterange(start_date, end_date):
    test_b[single_date.strftime("%d%b%Y")+'_B'] = train[single_date.strftime("%d%b%Y")+'_B']
    test_s[single_date.strftime("%d%b%Y")+'_S'] = train[single_date.strftime("%d%b%Y")+'_S']
    
test_s.head()

,issuer,Market,amtIssued,amtOutstanding,issued2,outstanding2,collateralType,collateral2,coupon,coupon2,...,31May2016_S,01Jun2016_S,02Jun2016_S,03Jun2016_S,04Jun2016_S,05Jun2016_S,06Jun2016_S,07Jun2016_S,08Jun2016_S,09Jun2016_S
0,490,2,183260000,183260000,3.358423e+16,3.358423e+16,19,361,10.000,100.000000,...,0,0,0,0,0,0,0,0,0,0
1,1193,2,261800000,198970000,6.853924e+16,3.958906e+16,26,676,5.500,30.250000,...,0,0,0,0,0,0,0,0,0,50000
2,2943,2,78540000,78540000,6.168532e+15,6.168532e+15,28,784,5.750,33.062500,...,0,0,0,0,0,0,0,0,0,0
3,3707,1,1570800000,1570800000,2.467413e+18,2.467413e+18,28,784,4.750,22.562500,...,0,210000,0,0,0,0,0,0,0,50000
4,3711,0,1309000000,1309000000,1.713481e+18,1.713481e+18,12,144,7.125,50.765625,...,0,0,0,0,0,0,0,0,0,0


In [129]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc.fit(train_b, train['09Jun2016_B'])
rfc1 = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc1.fit(train_s, train['09Jun2016_S'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=50,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [73]:
test_buy = rfc.predict(test_b)
test_sell = rfc1.predict(test_s)
fw = open('out4.csv', 'w')
w = csv.writer(fw)
w.writerow(('isin', 'buyvolume', 'sellvolume'))
fw1 = open('out4_r.csv', 'w')
w1 = csv.writer(fw1)
w1.writerow(('isin', 'buyvolume', 'sellvolume'))
for i in range(df.shape[0]):
    w.writerow((df['isin'][i], 3*test_buy[i], 3*test_sell[i]))
    w1.writerow((df['isin'][i], 3*test_sell[i], 3*test_buy[i]))
fw.close()
fw1.close()

In [75]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
gbr.fit(train_b, train['09Jun2016_B'])
gbr1 = GradientBoostingRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
gbr1.fit(train_s, train['09Jun2016_S'])

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=50, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [76]:
test_buy = gbr.predict(test_b)
test_sell = gbr1.predict(test_s)
fw = open('out6.csv', 'w')
w = csv.writer(fw)
w.writerow(('isin', 'buyvolume', 'sellvolume'))
fw1 = open('out6_r.csv', 'w')
w1 = csv.writer(fw1)
w1.writerow(('isin', 'buyvolume', 'sellvolume'))
for i in range(df.shape[0]):
    w.writerow((df['isin'][i], 3*test_buy[i], 3*test_sell[i]))
    w1.writerow((df['isin'][i], 3*test_sell[i], 3*test_buy[i]))
fw.close()
fw1.close()

In [172]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc.fit(train_b, train['09Jun2016_B'])
rfc1 = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc1.fit(train_s, train['09Jun2016_S'])

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_b, train['09Jun2016_B'])
lr1 = LinearRegression()
lr1.fit(train_s, train['09Jun2016_S'])

from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
gbr.fit(train_b, train['09Jun2016_B'])
gbr1 = GradientBoostingRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
gbr1.fit(train_s, train['09Jun2016_S'])

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=50, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [143]:
x = data[data['side']=='B']['volume'].min()
y = data[data['side']=='S']['volume'].min()

test_buy = lr.predict(test_b)
test_sell = lr1.predict(test_s)
test_buy1 = rfc.predict(test_b)
test_sell1 = rfc1.predict(test_s)
for i in range(df.shape[0]):
    if test_buy[i]<0:
        test_buy[i] = x
    if test_sell[i]<0:
        test_sell[i] = y
    if test_buy1[i]<0:
        test_buy1[i] = x
    if test_sell1[i]<0:
        test_sell1[i] = y

In [144]:
del train_b['16Mar2016_B'], train_s['16Mar2016_S']
train_b['09Jun2016_B'] = train['09Jun2016_B']
train_s['09Jun2016_S'] = train['09Jun2016_S']

train['10Jun2016_B'] = (2*test_buy+test_buy1)/3.0
train['10Jun2016_S'] = (2*test_sell+test_sell1)/3.0
del test_b['17Mar2016_B'], test_s['17Mar2016_S']
test_b['10Jun2016_B'] = train['10Jun2016_B']
test_s['10Jun2016_S'] = train['10Jun2016_S']

In [145]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc.fit(train_b, train['10Jun2016_B'])
rfc1 = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc1.fit(train_s, train['10Jun2016_S'])

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_b, train['10Jun2016_B'])
lr1 = LinearRegression()
lr1.fit(train_s, train['10Jun2016_S'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [146]:
x = data[data['side']=='B']['volume'].min()
y = data[data['side']=='S']['volume'].min()

test_buy = lr.predict(test_b)
test_sell = lr1.predict(test_s)
test_buy1 = rfc.predict(test_b)
test_sell1 = rfc1.predict(test_s)
for i in range(df.shape[0]):
    if test_buy[i]<0:
        test_buy[i] = x
    if test_sell[i]<0:
        test_sell[i] = y
    if test_buy1[i]<0:
        test_buy1[i] = x
    if test_sell1[i]<0:
        test_sell1[i] = y

In [147]:
del train_b['17Mar2016_B'], train_s['17Mar2016_S']
train_b['10Jun2016_B'] = train['10Jun2016_B']
train_s['10Jun2016_S'] = train['10Jun2016_S']

train['13Jun2016_B'] = (2*test_buy+test_buy1)/3.0
train['13Jun2016_S'] = (2*test_sell+test_sell1)/3.0
del test_b['18Mar2016_B'], test_s['18Mar2016_S']
test_b['13Jun2016_B'] = train['13Jun2016_B']
test_s['13Jun2016_S'] = train['13Jun2016_S']

In [148]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc.fit(train_b, train['13Jun2016_B'])
rfc1 = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=50)
rfc1.fit(train_s, train['13Jun2016_S'])

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_b, train['13Jun2016_B'])
lr1 = LinearRegression()
lr1.fit(train_s, train['13Jun2016_S'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [149]:
x = data[data['side']=='B']['volume'].min()
y = data[data['side']=='S']['volume'].min()

test_buy = lr.predict(test_b)
test_sell = lr1.predict(test_s)
test_buy1 = rfc.predict(test_b)
test_sell1 = rfc1.predict(test_s)
for i in range(df.shape[0]):
    if test_buy[i]<0:
        test_buy[i] = x
    if test_sell[i]<0:
        test_sell[i] = y
    if test_buy1[i]<0:
        test_buy1[i] = x
    if test_sell1[i]<0:
        test_sell1[i] = y

In [151]:
train['14Jun2016_B'] = (2*test_buy+test_buy1)/3.0
train['14Jun2016_S'] = (2*test_sell+test_sell1)/3.0

In [153]:
fw = open('out15.csv', 'w')
w = csv.writer(fw)
w.writerow(('isin', 'buyvolume', 'sellvolume'))
fw1 = open('out15_r.csv', 'w')
w1 = csv.writer(fw1)
w1.writerow(('isin', 'buyvolume', 'sellvolume'))
for i in range(df.shape[0]):
    w.writerow((df['isin'][i], train['10Jun2016_B'][i]+train['13Jun2016_B'][i]+train['14Jun2016_B'][i], train['10Jun2016_S'][i]+train['13Jun2016_S'][i]+train['14Jun2016_S'][i]))
    w1.writerow((df['isin'][i], train['10Jun2016_S'][i]+train['13Jun2016_S'][i]+train['14Jun2016_S'][i], train['10Jun2016_B'][i]+train['13Jun2016_B'][i]+train['14Jun2016_B'][i]))
fw.close()
fw1.close()

In [173]:
x = data[data['side']=='B']['volume'].min()
y = data[data['side']=='S']['volume'].min()

test_buy = lr.predict(test_b)
test_sell = lr1.predict(test_s)
test_buy1 = rfc.predict(test_b)
test_sell1 = rfc1.predict(test_s)
test_buy2 = gbr.predict(test_b)
test_sell2 = gbr1.predict(test_s)
fw = open('out16.csv', 'w')
w = csv.writer(fw)
w.writerow(('isin', 'buyvolume', 'sellvolume'))
fw1 = open('out16_r.csv', 'w')
w1 = csv.writer(fw1)
w1.writerow(('isin', 'buyvolume', 'sellvolume'))

for i in range(df.shape[0]):
    if i%1000==0:
        print i
    if test_buy[i]<0:
        test_buy[i] = x
    if test_sell[i]<0:
        test_sell[i] = y
    if test_buy1[i]<0:
        test_buy1[i] = x
    if test_sell1[i]<0:
        test_sell1[i] = y
    if test_buy2[i]<0:
        test_buy2[i] = x
    if test_sell2[i]<0:
        test_sell2[i] = y
    w.writerow((df['isin'][i], 3*(test_buy[i]+test_buy1[i]+test_buy2[i])/3.0, 3*(test_sell[i]+test_sell1[i]+test_sell2[i])/3.0))
    w1.writerow((df['isin'][i], 3*(test_sell[i]+test_sell1[i]+test_sell2[i])/3.0, 3*(test_buy[i]+test_buy1[i]+test_buy2[i])/3.0))
fw.close()
fw1.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
